In [20]:
import sys
import os
import pandas as pd

cwd = os.getcwd()
sys.path.append(cwd.replace('notebooks','') + 'DeltaStock')

from spark_deltalake.spark_operations import sparkDelta
from pyspark.sql.functions import col 


spark = sparkDelta()

hadoop-azure is available
azure-storage is available


In [21]:
""" available tables
ticker-sec
spy500
SEC_filings
company_facts
stock_prices
"""

path = spark.blobPath + '/company_facts'
#SEC_filings stock_prices 
spark.spark.sql(f'SELECT * FROM delta.`{path}` as f WHERE ticker = \'BXP\'').show()

+--------------------+--------------------+--------------------+----------+---------+--------------------+---+----+----------+-----+-----------+------------+------+
|             finType|               label|         description|       end|      val|     accessionNumber| fp|form|     filed|start|monthWindow|yearMonthDay|ticker|
+--------------------+--------------------+--------------------+----------+---------+--------------------+---+----+----------+-----+-----------+------------+------+
|AccountsPayableAn...|Accounts Payable ...|Carrying value as...|2008-12-31|1.71791E8|0001193125-09-167264| Q2|10-Q|2009-08-06|  NaN|        NaN|    20081231|   BXP|
|AccountsPayableAn...|Accounts Payable ...|Carrying value as...|2009-06-30|2.23909E8|0001193125-09-167264| Q2|10-Q|2009-08-06|  NaN|        NaN|    20090630|   BXP|
|AccountsPayableAn...|Accounts Payable ...|Sum of the carryi...|2008-12-31|1.71791E8|0001193125-09-225470| Q3|10-Q|2009-11-05|  NaN|        NaN|    20081231|   BXP|
|AccountsP

In [22]:
df = spark.get_spark_dataframe('SEC_filings', True, True)
df.head(5)

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription,fileURL,yearMonthDay,ticker
0,0000721371-23-000011,2023-02-16,2023-02-15,2023-02-16T16:16:45.000Z,,4,,,,4643,0,0,xslF345X03/wf-form4_167658218904919.xml,FORM 4,https://www.sec.gov/Archives/edgar/data/000072...,20230215,CAH
1,0000721371-23-000009,2023-02-16,2023-02-10,2023-02-16T16:11:15.000Z,,3,,,,7559,0,0,xslF345X02/wf-form3_167658185970869.xml,FORM 3,https://www.sec.gov/Archives/edgar/data/000072...,20230210,CAH
2,0000080255-23-002459,2023-02-14,,2023-02-14T12:37:58.000Z,34,SC 13G/A,005-34668,,,7100,0,0,cah13gadec22.txt,,https://www.sec.gov/Archives/edgar/data/000072...,20230214,CAH
3,0001104659-23-015555,2023-02-09,,2023-02-09T11:12:42.000Z,34,SC 13G/A,005-34668,23602686,,13028,0,0,tv0509-cardinalhealthinc.htm,SCHEDULE 13G/A,https://www.sec.gov/Archives/edgar/data/000072...,20230209,CAH
4,0000093751-23-000214,2023-02-06,,2023-02-06T09:43:33.000Z,34,SC 13G/A,005-34668,23588883,,8771,0,0,Cardinal_Health_Inc.txt,,https://www.sec.gov/Archives/edgar/data/000072...,20230206,CAH


In [23]:
spark.get_spark_dataframe('SEC_filings', True)
spark.get_spark_dataframe('stock_prices', True)
spark.get_spark_dataframe('company_facts', True)

DataFrame[finType: string, label: string, description: string, end: string, val: double, accessionNumber: string, fp: string, form: string, filed: string, start: string, monthWindow: string, yearMonthDay: string, ticker: string]

In [24]:
spark.spark.sql("""
                SELECT p.ticker, fintype, val, adjclose, fileURL FROM company_facts AS cf
                JOIN stock_prices as p ON cf.ticker = p.ticker
                JOIN SEC_filings as fil ON (fil.ticker = cf.ticker and fil.accessionNumber = cf.accessionNumber)
                WHERE cf.ticker = 'BXP'
                """).show()

+------+------------+---+-----------------+--------------------+
|ticker|     fintype|val|         adjclose|             fileURL|
+------+------------+---+-----------------+--------------------+
|   BXP|LineOfCredit|0.0|96.08738708496094|https://www.sec.g...|
|   BXP|LineOfCredit|0.0| 95.0941162109375|https://www.sec.g...|
|   BXP|LineOfCredit|0.0|94.50556182861328|https://www.sec.g...|
|   BXP|LineOfCredit|0.0|95.91079711914062|https://www.sec.g...|
|   BXP|LineOfCredit|0.0| 95.2044906616211|https://www.sec.g...|
|   BXP|LineOfCredit|0.0|94.54231262207031|https://www.sec.g...|
|   BXP|LineOfCredit|0.0|94.91754150390625|https://www.sec.g...|
|   BXP|LineOfCredit|0.0| 93.4386978149414|https://www.sec.g...|
|   BXP|LineOfCredit|0.0|94.75568389892578|https://www.sec.g...|
|   BXP|LineOfCredit|0.0|95.82252502441406|https://www.sec.g...|
|   BXP|LineOfCredit|0.0|96.78633880615234|https://www.sec.g...|
|   BXP|LineOfCredit|0.0|97.00704193115234|https://www.sec.g...|
|   BXP|LineOfCredit|0.0|

In [25]:
loadedTickers = spark.get_min_max_fillings_date_filter_ticker()
loadedTickers.head()

,ticker,max_time,min_time
0,BXP,2024-08-26,2014-12-31
1,CAH,2024-08-21,2012-08-31
2,CDNS,2024-08-28,2017-02-08
3,CHRW,2024-08-27,2011-04-29
4,COF,2024-08-14,2018-08-14


In [26]:
spark.sparkStop()